In [309]:
# imports
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import json

In [310]:
embeddings_path = "../data/KTAS_DATA.csv"
KTAS_df = pd.read_csv(embeddings_path, encoding='cp949')

KTAS_df.head()

,Index,KTAS1_Group,KTAS1_Code,KTAS2_Group,KTAS2_Code,KTAS3_Group,KTAS3_Code,KTAS4_Group,KTAS4_Code,KTAS_CODE,KTAS_Level
0,1,성인,A,물질오용,A,물질오용/중독,A,중증 호흡곤란,AA,AAAAA,1
1,2,성인,A,물질오용,A,물질오용/중독,A,쇼크,AB,AAAAB,1
2,3,성인,A,물질오용,A,물질오용/중독,A,무의식(GCS 3-8),AC,AAAAC,1
3,4,성인,A,물질오용,A,물질오용/중독,A,중등도 호흡곤란,AD,AAAAD,2
4,5,성인,A,물질오용,A,물질오용/중독,A,혈역학적 장애,AE,AAAAE,2


# 환자 정보 확인

In [311]:
patient_data = dict()
input_data = input("""
환자의 정보를 입력해주세요.
성별, 연령, 증상, 기저질환, 의식 수준, 혈압, 맥박, 호흡 수, 체온
예) 남성, 60, 쥐어짜는듯한 가슴 통증, 없음, 9, 120/80, 110, 16, 36.8
""").split(',')

patient_data["성별"] = input_data[0]
patient_data["연령"] = input_data[1]
patient_data["증상"] = input_data[2]
patient_data["기저질환"] = input_data[3]
patient_data["의식수준"] = input_data[4]
patient_data["혈압"] = input_data[5]
patient_data["맥박"] = input_data[6]
patient_data["호흡 수"] = input_data[7]
patient_data["체온"] = input_data[8]

json.dumps(patient_data,ensure_ascii=False)


환자의 정보를 입력해주세요.
성별, 연령, 증상, 기저질환, 의식 수준, 혈압, 맥박, 호흡 수, 체온
예) 남성, 60, 쥐어짜는듯한 가슴 통증, 없음, 9, 120/80, 110, 16, 36.8
여성, 80, 쥐어짜는듯한 가슴 통증, 고혈압, 12, 150/95, 88, 16, 37.8


'{"성별": "여성", "연령": " 80", "증상": " 쥐어짜는듯한 가슴 통증", "기저질환": " 고혈압", "의식수준": " 12", "혈압": " 150/95", "맥박": " 88", "호흡 수": " 16", "체온": " 37.8"}'

# KTAS_1단계 검증

In [312]:
if int(patient_data["연령"]) >= 15:
    KTAS1_df = df[df['KTAS1_Code'] == 'A']
else:
    KTAS1_df = df[df['KTAS1_Code'] == 'B']
    
group2 = KTAS1_df['KTAS2_Group'].unique().tolist()
print(group2)

['물질오용', '정신건강', '신경과', '안과', '코질환', '귀질환', '이비인후과-입,인후,경부', '호흡기', '심혈관', '소화기', '산부인과', '비뇨기과', '정형외과', '외상', '환경손상', '피부', '일반']


# KTAS_2단계 검증

In [313]:
KTAS2_Question = '''
너는 응급 환자의 중증도를 판단해야하는 구급대원이야.
한국의 응급의료 분야에서 사용되는 KTAS(Korean Triage and Acuity Scale) 분류 체계에 따라 분류해야해.

입력값: 환자 정보, KTAS 옵션

결과값: option, probability, reason을 포함하는 JSON 객체
- option: 가장 가능성이 높은 KTAS 옵션
- probability: 해당 KTAS 옵션이 맞을 확률 (0<=probability<=1)
- reason: 선정 이유 (자세하게 설명할 것)


주의 사항:
- 만약 주어진 정보 안에서 가능성이 높은 옵션이 없다면, "환자 증상을 더 자세히 작성해주세요"를 출력할 것
'''

In [314]:
KTAS2_Response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": KTAS2_Question
    },
    {
      "role": "user",
      "content": "환자 정보는 다음과 같습니다.\n" + json.dumps(patient_data,ensure_ascii=False)
    },
    {
      "role": "user",
      "content": "KTAS 내 주증상 군은 다음과 같습니다. \n" + ', '.join(group2)
    }
  ],
  temperature=0,
  max_tokens=3000
)
print(KTAS2_Response.get("choices")[0].get("message").get("content"))
KTAS2_ANSWER = json.loads(KTAS2_Response.get("choices")[0].get("message").get("content"))
print(KTAS2_ANSWER)

{"option": "심혈관", "probability": 0.9, "reason": "환자의 주증상인 쥐어짜는듯한 가슴 통증과 기저질환인 고혈압을 고려할 때, 심혈관 관련 질환으로 분류될 가능성이 높습니다. 또한, 환자의 연령이 80세로 고령이므로 심혈관 질환 발생 가능성이 높아집니다. 혈압이 150/95로 상승하였으며 맥박이 88으로 비정상적인 수치를 보이고 있습니다. 이러한 이유로 심혈관 질환으로 분류될 가능성이 높습니다."}
{'option': '심혈관', 'probability': 0.9, 'reason': '환자의 주증상인 쥐어짜는듯한 가슴 통증과 기저질환인 고혈압을 고려할 때, 심혈관 관련 질환으로 분류될 가능성이 높습니다. 또한, 환자의 연령이 80세로 고령이므로 심혈관 질환 발생 가능성이 높아집니다. 혈압이 150/95로 상승하였으며 맥박이 88으로 비정상적인 수치를 보이고 있습니다. 이러한 이유로 심혈관 질환으로 분류될 가능성이 높습니다.'}


In [315]:
print(KTAS2_ANSWER['option'])

심혈관


# KTAS_3단계 검증

In [316]:
# 가장 가능성 높은 KTAS2 분류로 필터링한다.
KTAS2_df = KTAS1_df[KTAS1_df['KTAS2_Group'] == KTAS2_ANSWER['option']]
KTAS2_df.head()

group3 = KTAS2_df['KTAS3_Group'].unique().tolist()
print(group3)

['심정지(비외상성)', '심정지(외상성)', '흉통(심인성)', '흉통(비 심인성)', '심계항진/불규칙 심장박동', '고혈압', '전신쇠약', '실신/전-실신', '전신성부종', '양측성 다리부기/부종', '사지 말단부위에 맥박이 없고 차가움', '일측성의 홍조 띈 뜨거운 사지']


In [317]:
KTAS3_Question = f'''
너는 응급 환자의 중증도를 판단해야하는 구급대원이야.
한국의 응급의료 분야에서 사용되는 KTAS(Korean Triage and Acuity Scale) 분류 체계에 따라 분류해야해.

입력값: 환자 정보, KTAS 1차 고려사항 옵션

결과값: option, probability, reason을 포함하는 JSON 객체
- option: 가장 가능성이 높은 KTAS 1차 고려사항 옵션
- probability: 해당 KTAS 1차 고려사항이 맞을 확률 (0<=probability<=1)
- reason: 선정 이유 (자세하게 설명할 것)

주의 사항:
- 만약 주어진 정보 안에서 가능성이 높은 옵션이 없다면, "환자 증상을 더 자세히 작성해주세요"를 출력할 것
'''

In [318]:
KTAS3_Response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": KTAS3_Question
    },
    {
      "role": "user",
      "content": "환자 정보는 다음과 같습니다.\n" + json.dumps(patient_data,ensure_ascii=False)
    },
    {
      "role": "user",
      "content": f"{KTAS2_ANSWER['option']}의 1차 고려사항은 다음과 같습니다. \n" + ', '.join(group3)
    }
  ],
  temperature=0,
  max_tokens=3000
)
print(KTAS3_ANSWER)
KTAS3_ANSWER = json.loads(KTAS3_Response.get("choices")[0].get("message").get("content"))
print(KTAS3_ANSWER)

{'option': '열', 'probability': 0.8, 'reason': '환자의 체온이 37.3도로 정상체온 범위를 넘어섰습니다. 또한, 환자의 증상으로는 목 통증이 있습니다. 목 통증은 일반적으로 열과 관련이 있을 수 있으며, 열은 전염성 질환의 증상일 수 있습니다. 따라서, 열이 가장 가능성이 높은 1차 고려사항입니다.'}
{'option': '흉통(심인성)', 'probability': 0.8, 'reason': '환자의 주요 증상인 쥐어짜는듯한 가슴 통증과 함께 고혈압, 심장 박동 수의 증가가 관찰되었습니다. 이는 심혈관 질환과 관련된 심인성 흉통의 가능성을 높입니다. 추가적인 정보를 수집하여 KTAS 1차 고려사항을 최종적으로 판단해야 합니다.'}


# KTAS_4단계 검증

In [319]:
# 가장 가능성 높은 KTAS2 분류로 필터링한다.
KTAS3_df = KTAS2_df[KTAS2_df['KTAS3_Group'] == KTAS3_ANSWER['option']]

group4 = KTAS3_df['KTAS4_Group'].unique().tolist()
print(group4)

['중증 호흡곤란', '쇼크', '무의식(GCS 3-8)', '중등도 호흡곤란', '혈역학적 장애', '의식변화(GCS 9-13)', '열,면역억제 상태', '패혈증 의증(3 SIRS 기준만족)', '급성 중심성 중증 통증(8-10)', '심인성 통증']


In [320]:
KTAS4_Question = f'''
너는 응급 환자의 중증도를 판단해야하는 구급대원이야.
한국의 응급의료 분야에서 사용되는 KTAS(Korean Triage and Acuity Scale) 분류 체계에 따라 분류해야해.

입력값: 환자 정보, KTAS 2차 고려사항 옵션

결과값: option, probability, reason을 포함하는 JSON 객체
- option: 가장 가능성이 높은 KTAS 2차 고려사항 옵션
- probability: 해당 KTAS 2차 고려사항이 맞을 확률 (0<=probability<=1)
- reason: 선정 이유 (자세하게 설명할 것)

주의 사항:
- 만약 주어진 정보 안에서 가능성이 높은 옵션이 없다면, "환자 증상을 더 자세히 작성해주세요"를 출력할 것
'''

In [321]:
KTAS4_Response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": KTAS3_Question
    },
    {
      "role": "user",
      "content": "환자 정보는 다음과 같습니다.\n" + json.dumps(patient_data,ensure_ascii=False)
    },
    {
      "role": "user",
      "content": f"현재 {KTAS2_ANSWER['option']} 중 {KTAS3_ANSWER['option']}으로 판단하고 있습니다."
    },
    {
      "role": "user",
      "content": "가능한 2차 고려사항은 다음과 같습니다. \n" + ', '.join(group4)
    }
  ],
  temperature=0,
  max_tokens=3000
)
print(KTAS4_ANSWER)
KTAS4_ANSWER = json.loads(KTAS4_Response.get("choices")[0].get("message").get("content"))
print(KTAS4_ANSWER)

{'option': '열', 'probability': 0.8, 'reason': '환자의 체온이 37.3도로 정상 범위를 넘어섰으며, 목 통증과 함께 증상이 나타나 있습니다. 따라서 열이 가장 가능성이 높은 2차 고려사항입니다.'}
{'option': '중증 호흡곤란', 'probability': 0.8, 'reason': '환자의 증상으로 쥐어짜는 듯한 가슴 통증이 있으며, 혈압이 상승하고 호흡수가 정상 범위에 있지만 의식 수준이 낮아 중증 호흡곤란이 의심됩니다. 이는 심혈관 중 흉통으로 인한 심장 기능 저하로 인해 발생할 수 있습니다.'}


# 최종 결과 도출

In [322]:
KTAS_LEVEL = {
    1 : "즉각적인 처치가 필요하며 생명이나 사지를 위협하는(또는 악화 가능성이 높은) 상태",
    2 : "생명 혹은 사지, 신체기능에 잠재적인 위협이 있으며 이에 대한 빠른 치료가 필요한 경우",
    3 : "치료가 필요한 상태로 진행할 수도 있는 잠재적 가능성을 고려해야 하는 경우",
    4 : "환자의 나이, 통증이나 악화/합병증에 대한 가능성을 고려할 때 1~2시간 안에 처치나 재평가를 시행하면 되는 상태",
    5 : "긴급하지만 응급은 아닌 상태, 만성적인 문제로 인한 것이거나, 악화의 가능성이 낮은 상태"
}

In [323]:
# 가장 가능성 높은 KTAS2 분류로 필터링한다.
KTAS4_df = KTAS3_df[KTAS3_df['KTAS4_Group'] == KTAS4_ANSWER['option']]

KTAS = KTAS4_df.iloc[0].to_dict()

KTAS

FINAL_MESSAGE = f'''
응급 환자 상태: {json.dumps(patient_data,ensure_ascii=False)}

KTAS 분류 내용: {KTAS['KTAS_CODE']} : {KTAS['KTAS2_Group']} - {KTAS['KTAS3_Group']} - {KTAS['KTAS4_Group']}

KTAS 분류 등급: {KTAS['KTAS_Level']}등급: {KTAS_LEVEL[KTAS['KTAS_Level']]}

KTAS 판단 근거1: {KTAS2_ANSWER['reason']}

KTAS 판단 근거2: {KTAS3_ANSWER['reason']}

KTAS 판단 근거3: {KTAS4_ANSWER['reason']}
'''

print(FINAL_MESSAGE)


응급 환자 상태: {"성별": "여성", "연령": " 80", "증상": " 쥐어짜는듯한 가슴 통증", "기저질환": " 고혈압", "의식수준": " 12", "혈압": " 150/95", "맥박": " 88", "호흡 수": " 16", "체온": " 37.8"}

KTAS 분류 내용: AICAA : 심혈관 - 흉통(심인성) - 중증 호흡곤란

KTAS 분류 등급: 1등급: 즉각적인 처치가 필요하며 생명이나 사지를 위협하는(또는 악화 가능성이 높은) 상태

KTAS 판단 근거1: 환자의 주증상인 쥐어짜는듯한 가슴 통증과 기저질환인 고혈압을 고려할 때, 심혈관 관련 질환으로 분류될 가능성이 높습니다. 또한, 환자의 연령이 80세로 고령이므로 심혈관 질환 발생 가능성이 높아집니다. 혈압이 150/95로 상승하였으며 맥박이 88으로 비정상적인 수치를 보이고 있습니다. 이러한 이유로 심혈관 질환으로 분류될 가능성이 높습니다.

KTAS 판단 근거2: 환자의 주요 증상인 쥐어짜는듯한 가슴 통증과 함께 고혈압, 심장 박동 수의 증가가 관찰되었습니다. 이는 심혈관 질환과 관련된 심인성 흉통의 가능성을 높입니다. 추가적인 정보를 수집하여 KTAS 1차 고려사항을 최종적으로 판단해야 합니다.

KTAS 판단 근거3: 환자의 증상으로 쥐어짜는 듯한 가슴 통증이 있으며, 혈압이 상승하고 호흡수가 정상 범위에 있지만 의식 수준이 낮아 중증 호흡곤란이 의심됩니다. 이는 심혈관 중 흉통으로 인한 심장 기능 저하로 인해 발생할 수 있습니다.

